# Creator: Ivan Bardarov <br> (University of Strathclyde, March 2019)
## This module sets up the server and creates REST endpoints


In [ ]:
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles
import sys
sys.path.append('../recommender/')
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio
from io import BytesIO
from pathlib import Path
from cloth_detector import *
from style_recommender import *
import email_agent as ea

# application init
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])

# expose the files in these folders to be allowed for acess
app.mount('/static', StaticFiles(directory='../static'))
app.mount('/static', StaticFiles(directory='../static/image_data'))
app.mount('/static', StaticFiles(directory='../static/tmp'))
app.mount('/static', StaticFiles(directory='../static/assets'))

# define JSON keys
ORIGINAL = "original_image"
CROPPED = "cropped_items"
CATS = "categories"
CHOSEN_IMAGE = "chosen_image"
CHOSEN_CAT = "chosen_cat"
ID2CAT = "id2cat"

@app.route('/')
def index(request):
    """
    An index API which returns the main webapge.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    HTMLResponse
        the requested html file

    """
    html = Path("../view/index.html")
    return HTMLResponse(html.open().read())


@app.route('/email', methods=['POST'])
async def send_email(request):
    """
    This function handles POST request sent to /email.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        Empty JSON response to finish handling the request

    """
    json = await request.json()
    #pass the request along
    await ea.send_email(json["name"], json["email"], json["comments"])
    print(json)
    return JSONResponse({})


@app.route('/analyze', methods=['POST'])
async def detect_items(request):
    """
    This function handles POST request sent to /analyze.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        a json object with the results array

    """
    request_json = await request.json()
    #pass the request along
    results_arr = get_recommendations(request_json[CHOSEN_IMAGE], request_json[CHOSEN_CAT])
    print(results_arr)
    return JSONResponse({"results":results_arr})


@app.route('/detect', methods=['POST'])
async def analyze(request):
    """
    This function handles POST request sent to /detect.

    Parameters
    ----------
    request : Request
        the received request
        
    Returns
    -------
    JSONResponse
        an array of result image paths

    """
    data = await request.form()
    #read the file
    img_bytes = await (data['file'].read())
    #pass the request along
    i_fns, cat_list, id2cat = get_detected_items(Image.open(BytesIO(img_bytes)))
    #construct the JSON response
    response = {CROPPED: i_fns,
                CATS: cat_list,
                ID2CAT: id2cat}
    print(response)
    return JSONResponse(response)


if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)

INFO: Started server process [18919]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:1997 (Press CTRL+C to quit)
INFO: ('31.205.68.60', 34330) - "GET / HTTP/1.1" 200
INFO: ('31.205.68.60', 34332) - "POST /detect HTTP/1.1" 200
INFO: ('31.205.68.60', 34332) - "GET /static/tmp/b2b19f72-1a5b-4bdf-a58f-407231213cb9.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34336) - "GET /static/tmp/57495482-7bbf-4306-b674-c5f7ccaa9ba8.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34334) - "GET /static/tmp/475fc877-e1fb-43be-a2e9-38c17d095498.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34340) - "GET /static/tmp/4847ceaf-22f1-4346-8da2-fa7e86a9de7c.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34338) - "GET /static/tmp/931ce2ee-efe9-4846-bf35-69b5eba72e78.jpg HTTP/1.1" 200


{'cropped_items': ['../static/tmp/b2b19f72-1a5b-4bdf-a58f-407231213cb9.jpg', '../static/tmp/57495482-7bbf-4306-b674-c5f7ccaa9ba8.jpg', '../static/tmp/475fc877-e1fb-43be-a2e9-38c17d095498.jpg', '../static/tmp/4847ceaf-22f1-4346-8da2-fa7e86a9de7c.jpg', '../static/tmp/931ce2ee-efe9-4846-bf35-69b5eba72e78.jpg'], 'categories': [3, 4, 7, 8, 8], 'id2cat': ['bag', 'belt', 'boots', 'footwear', 'outer', 'dress', 'sunglasses', 'pants', 'top', 'shorts', 'skirt', 'headwear', 'scarf/tie']}


INFO: 4.091: POST https://background-removal.rorocloud.io/predict
INFO: ('31.205.68.60', 34336) - "POST /analyze HTTP/1.1" 200
INFO: ('31.205.68.60', 34336) - "GET /static/image_data/TS02G02MBLK_M_1.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34342) - "GET /static/image_data/vice-high-waisted-slash-knee-skinny-jeans-black.jpeg HTTP/1.1" 200


['../static/image_data/TS02G02MBLK_M_1.jpg', '../static/image_data/vice-high-waisted-slash-knee-skinny-jeans-black.jpeg', '../static/image_data/black-vice-high-waisted-coated-skinny-jeans.jpg', '../static/image_data/blue-vice-high-waisted-raw-hem-skinny-jeans.jpeg']


INFO: ('31.205.68.60', 34336) - "GET /static/image_data/black-vice-high-waisted-coated-skinny-jeans.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34336) - "GET /static/image_data/blue-vice-high-waisted-raw-hem-skinny-jeans.jpeg HTTP/1.1" 200
INFO: 3.905: POST https://background-removal.rorocloud.io/predict
INFO: ('31.205.68.60', 34356) - "POST /analyze HTTP/1.1" 200


['../static/image_data/TS02G02MBLK_M_1.jpg', '../static/image_data/vice-high-waisted-slash-knee-skinny-jeans-black.jpeg', '../static/image_data/black-vice-high-waisted-coated-skinny-jeans.jpg', '../static/image_data/blue-vice-high-waisted-raw-hem-skinny-jeans.jpeg']


INFO: ('31.205.68.60', 34358) - "POST /detect HTTP/1.1" 200
INFO: ('31.205.68.60', 34358) - "GET /static/tmp/6b6cf114-bac0-49a4-b40d-fcb91fbe921d.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34368) - "GET /static/tmp/c1a9fdc3-013c-467d-acfd-ec9be4c95c92.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34362) - "GET /static/tmp/f6cda804-ed6b-44cf-b134-82b3e1fd9017.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34360) - "GET /static/tmp/73297982-7e76-4b82-a81d-e6d3fcdedd9b.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34366) - "GET /static/tmp/7db25e88-6931-4994-9609-0a983cbeeaeb.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34364) - "GET /static/tmp/21b252e3-68b0-412a-8415-17343a9b7a99.jpg HTTP/1.1" 200
INFO: ('31.205.68.60', 34358) - "GET /static/tmp/5a30c690-c6dd-4356-b497-3ec6d581809d.jpg HTTP/1.1" 200


{'cropped_items': ['../static/tmp/6b6cf114-bac0-49a4-b40d-fcb91fbe921d.jpg', '../static/tmp/73297982-7e76-4b82-a81d-e6d3fcdedd9b.jpg', '../static/tmp/f6cda804-ed6b-44cf-b134-82b3e1fd9017.jpg', '../static/tmp/21b252e3-68b0-412a-8415-17343a9b7a99.jpg', '../static/tmp/7db25e88-6931-4994-9609-0a983cbeeaeb.jpg', '../static/tmp/5a30c690-c6dd-4356-b497-3ec6d581809d.jpg', '../static/tmp/c1a9fdc3-013c-467d-acfd-ec9be4c95c92.jpg'], 'categories': [0, 3, 3, 4, 4, 8, 10], 'id2cat': ['bag', 'belt', 'boots', 'footwear', 'outer', 'dress', 'sunglasses', 'pants', 'top', 'shorts', 'skirt', 'headwear', 'scarf/tie']}


INFO: ('31.205.68.60', 34372) - "GET / HTTP/1.1" 200
